In [1]:
%matplotlib inline

In [2]:
import cv2
import numpy as np
import glob
import os
from skimage import io
from sklearn.cross_validation import train_test_split
import matplotlib.pyplot as plt

/Users/anishagartia/anaconda/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Convolution2D, LocallyConnected2D, MaxPooling2D, Flatten
from keras.optimizers import  SGD
import keras.backend as k
from keras.callbacks import EarlyStopping

Using Theano backend.


In [4]:
data_path = '../data/'; 
train_path_pos = os.path.join(data_path, 'caltech_faces/Caltech_CropFaces'); #Positive training examples. 36x36 head crops
non_face_scn_path = os.path.join(data_path, 'train_non_face_scenes'); #We can mine random or hard negatives from here
test_scn_path = os.path.join(data_path,'test_scenes/test_jpg'); #CMU+MIT test scenes
# test_scn_path = fullfile(data_path,'extra_test_scenes'); #Bonus scenes
label_path = os.path.join(data_path,'test_scenes/ground_truth_bboxes.txt'); #the ground truth face locations in the test set
        
TrainPosPaths = []
for i in glob.glob(os.path.join(train_path_pos, '*.jpg')):
    TrainPosPaths.append(i)
    
TrainNegPaths = []
for i in glob.glob(os.path.join(non_face_scn_path, '*.jpg')):
    TrainNegPaths.append(i)    
    
TestPaths = []
for i in glob.glob(os.path.join(test_scn_path, '*.jpg')):
    TestPaths.append(i)    

print('Getting Train data...')    
X_PosTrain = []   
Label_PosTrain = []
for i in TrainPosPaths:
    imgRead = io.imread(i,as_grey=True)
    X_PosTrain.append(imgRead)        
    Label_PosTrain.append(1)  
X_PosTrain = np.asarray(X_PosTrain)  
    
numSamples = 10000
numImg = len(TrainNegPaths)
numPatchPerImg = np.int(numSamples/numImg)
X_NegTrain = []
Label_NegTrain = []
for i in TrainNegPaths:
    imgRead = io.imread(i,as_grey=True)
    randx = np.random.randint(0,imgRead.shape[1]-36, numPatchPerImg)
    randy = np.random.randint(0,imgRead.shape[0]-36, numPatchPerImg)
    for j in range(0,numPatchPerImg):
        X_NegTrain.append(imgRead[randy[j]:randy[j]+36,randx[j]:randx[j]+36])    
        Label_NegTrain.append(0)
X_NegTrain = np.asarray(X_NegTrain)

X_Train = np.vstack((X_PosTrain,X_NegTrain))
X_Train = np.reshape(X_Train, (X_Train.shape[0],1, X_Train.shape[1], X_Train.shape[2]))
Y_Train = np.hstack((Label_PosTrain,Label_NegTrain))

X_Train = X_Train.astype('float32')
X_Train /= 255

#nb_classes = 2
Y_Train = np.array(Y_Train)
#Y_Train = np_utils.to_categorical(Y_Train, nb_classes)


# print('Getting Test data...')  
# X_Test = []   
# for i in TestPaths:
#     imgRead = io.imread(i,as_grey=True)
#     X_Test.append(imgRead)     
# X_Test = np.asarray(X_Test) 
# X_Test = np.reshape(X_Test, (X_Test.shape[0],1, X_Test.shape[1], X_Test.shape[2]))

print('Completed')

Getting Train data...
Completed


In [32]:
print('Creating the Model')
weight_init="he_uniform"
activation="relu"
border_mode="valid"
nb_filter=[10,10,10,10,10]
nb_row=[3,3,3,3,3]
nb_col=[3,3,3,3,3]
output_fc=[36,36]
pool_size=(3,3)
stride_size=(2,2)
dropout_percent=0.5
learning_rate=1e-3
mommentum_update=0.9
decay_rate=1e-6
training_batch_size=10
val_batch_size=10
no_epoch=1000
no_of_channels = 1
img_row = 36
img_col = 36
nb_classes = 1
loss="binary_crossentropy"

Creating the Model


DeepFaces (https://research.facebook.com/publications/deepface-closing-the-gap-to-human-level-performance-in-face-verification/)

In [33]:
model = Sequential()
#Deepnet Archictecture
#c1
model.add(Convolution2D(nb_filter[0], nb_row[0], nb_col[0], border_mode=border_mode, input_shape=(no_of_channels,img_row,img_col),init=weight_init))
model.add(Activation(activation))
#m2
model.add(MaxPooling2D(pool_size = pool_size, strides = stride_size))
#c3
model.add(Convolution2D(nb_filter[1], nb_row[1], nb_col[1], border_mode=border_mode, init=weight_init))
model.add(Activation(activation))
#l4
model.add(LocallyConnected2D(nb_filter[2], nb_row[2], nb_col[2],border_mode=border_mode, init=weight_init))
model.add(Activation(activation))
# #l5
model.add(LocallyConnected2D(nb_filter[3], nb_row[3], nb_col[3],border_mode=border_mode, init=weight_init))
model.add(Activation(activation))
#L6
model.add(LocallyConnected2D(nb_filter[4], nb_row[4], nb_col[4],border_mode=border_mode, init=weight_init))
#Dropout
model.add(Dropout(dropout_percent))
model.add(Flatten())
#F7
model.add(Dense(output_fc[0],init=weight_init))
model.add(Activation(activation)) 
#normalization l2 
#F8
# model.add(Dense(output_fc[1],init=weight_init))
model.add(Dense(nb_classes, activation='sigmoid',init=weight_init))
#learning process
sgd = SGD(lr=learning_rate, decay=1e-6, momentum=0.9, nesterov=True)
    
    

In [7]:
model.compile(loss=loss,optimizer='rmsprop',metrics=['accuracy'])
    

In [34]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
convolution2d_1 (Convolution2D)  (None, 10, 34, 34)    100         convolution2d_input_1[0][0]      
____________________________________________________________________________________________________
activation_4 (Activation)        (None, 10, 34, 34)    0           convolution2d_1[0][0]            
____________________________________________________________________________________________________
maxpooling2d_1 (MaxPooling2D)    (None, 10, 16, 16)    0           activation_4[0][0]               
____________________________________________________________________________________________________
convolution2d_2 (Convolution2D)  (None, 10, 14, 14)    910         maxpooling2d_1[0][0]             
___________________________________________________________________________________________

In [8]:
earlystopping = EarlyStopping(monitor='val_loss', min_delta=1e-6, patience=5, verbose=0, mode='auto')
model.fit(X_Train, Y_Train, batch_size=training_batch_size,  nb_epoch=no_epoch, callbacks=[earlystopping], validation_split=0.2)
#score = model.evaluate(X_Test, Y_Test, batch_size=val_batch_size)

Model with Only Dense Layer

In [10]:
X_Train = X_Train.reshape(X_Train.shape[0],-1)
X_Train.shape

(16577, 1296)

In [ ]:
model.save('nn_model.h5', overwrite=True)

In [11]:
# model = Sequential()
# model.add(Dense(512,input_shape=(X_Train.shape[1],)))
# model.add(Activation('sigmoid'))
# model.add(Dropout(0.2))
# model.add(Dense(512))
# model.add(Activation('sigmoid'))
# model.add(Dropout(0.2))
# model.add(Dense(nb_classes))
# model.add(Activation('sigmoid'))

In [12]:
# model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
dense_1 (Dense)                  (None, 512)           664064      dense_input_1[0][0]              
____________________________________________________________________________________________________
activation_1 (Activation)        (None, 512)           0           dense_1[0][0]                    
____________________________________________________________________________________________________
dropout_1 (Dropout)              (None, 512)           0           activation_1[0][0]               
____________________________________________________________________________________________________
dense_2 (Dense)                  (None, 512)           262656      dropout_1[0][0]                  
___________________________________________________________________________________________

In [13]:
# learning_rate=5e-3
# sgd = SGD(lr=learning_rate, decay=1e-5, momentum=0.9, nesterov=True)
# model.compile(loss='binary_crossentropy', optimizer=sgd, metrics=['accuracy'])

In [14]:
# earlystopping = EarlyStopping(monitor='val_loss', min_delta=1.05e-4, patience=5, verbose=0, mode='auto')
# model.fit(X_Train, Y_Train, batch_size=32, nb_epoch=25, verbose=1, validation_split=0.2)

Train on 13261 samples, validate on 3316 samples
Epoch 1/25
13261/13261 [==============================] - 6s - loss: 0.1207 - acc: 0.9687 - val_loss: 0.0077 - val_acc: 1.0000
Epoch 2/25
13261/13261 [==============================] - 6s - loss: 0.0077 - acc: 0.9995 - val_loss: 0.0035 - val_acc: 1.0000
Epoch 3/25
13261/13261 [==============================] - 6s - loss: 0.0038 - acc: 0.9998 - val_loss: 0.0019 - val_acc: 1.0000
Epoch 4/25
13261/13261 [==============================] - 9s - loss: 0.0026 - acc: 0.9998 - val_loss: 0.0015 - val_acc: 1.0000
Epoch 5/25
13261/13261 [==============================] - 9s - loss: 0.0022 - acc: 0.9998 - val_loss: 0.0012 - val_acc: 1.0000
Epoch 6/25
13261/13261 [==============================] - 9s - loss: 0.0017 - acc: 0.9998 - val_loss: 7.6938e-04 - val_acc: 1.0000
Epoch 7/25
13261/13261 [==============================] - 10s - loss: 0.0013 - acc: 0.9999 - val_loss: 7.9499e-04 - val_acc: 1.0000
Epoch 8/25
13261/13261 [=============================

In [15]:
# model.save('nn_model.h5', overwrite=True)

In [31]:
# import pydot
# from keras.utils.visualize_util import plot
# plot(model, to_file='model.png')

RuntimeError: Failed to import pydot. You must install pydot and graphviz for `pydotprint` to work.